###  1、流水数据文件夹下的三张表合并&去重

#### 为什么要把流水数据合起来用？事实证明就算是把流水数据下的三张表都用上了，也有一部分的merchant_id找不到任何的流水记录，我们当然希望尽可能地不遗漏所有的流水信息，而三张表的信息量肯定比一张表大（注意去重交易id）

In [ ]:
import pandas as pd

# 文件路径
file_paths = [
    '流水数据/table_lend_transaction_final.txt',
    '流水数据/table_loss_transaction_testdata.txt',
    '流水数据/table_loss_transaction_trainingdata.txt'
]

# 读取数据
dataframes = [pd.read_csv(file) for file in file_paths]

# 打印各个文件的数据量
for df, path in zip(dataframes, file_paths):
    print(f"{path} 的数据量: {df.shape[0]}")

# 合并数据以寻找重复项
combined_df = pd.concat(dataframes, ignore_index=True)

# 查找完全一致的重复行
duplicates = combined_df[combined_df.duplicated(keep=False)]
unique_duplicates = duplicates.drop_duplicates()

# 打印完全一致的重复数据量
print(f"完全一致的重复数据量: {unique_duplicates.shape[0]}")

# 删除完全一致的重复行，只保留一个
combined_df = combined_df.drop_duplicates()

# 打印去重后的总数据量
print(f"去重后的总数据量: {combined_df.shape[0]}")

# 如果需要，可以保存合并后的数据
# combined_df.to_csv('流水数据/combined_data_cleaned.txt', index=False)

流水数据/table_lend_transaction_final.txt 的数据量: 3590059
流水数据/table_loss_transaction_testdata.txt 的数据量: 2404994
流水数据/table_loss_transaction_trainingdata.txt 的数据量: 2309665
完全一致的重复数据量: 940647
去重后的总数据量: 7295027


In [ ]:
import pandas as pd

# 文件路径
file_paths = [
    '流水数据/table_lend_transaction_final.txt',
    '流水数据/table_loss_transaction_testdata.txt',
    '流水数据/table_loss_transaction_trainingdata.txt'
]

# 读取数据并打印各个文件的数据量
dataframes = []
for file in file_paths:
    df = pd.read_csv(file)
    print(f"{file} 的数据量为: {df.shape[0]}行")
    dataframes.append(df)

# 合并数据
combined_df = pd.concat(dataframes, ignore_index=True)
print(f"合并后的总数据量为: {combined_df.shape[0]}行")

# 保存合并后的数据到新文件
combined_df.to_csv('流水数据/combined_data.txt', index=False)


流水数据/table_lend_transaction_final.txt 的数据量为: 3590059行
流水数据/table_loss_transaction_testdata.txt 的数据量为: 2404994行
流水数据/table_loss_transaction_trainingdata.txt 的数据量为: 2309665行
合并后的总数据量为: 8304718行


In [ ]:
import pandas as pd

# 文件路径
file_path = '流水数据/combined_data.txt'

# 读取数据
df = pd.read_csv(file_path)

# 原始数据量打印
print(f"原始数据量: {df.shape[0]}行")

# 删除重复行，只保留一个
df = df.drop_duplicates()

# 打印去重后的数据量
print(f"去重后的数据量: {df.shape[0]}行")

# 保存去重后的数据到新文件
df.to_csv('流水数据/combined_data_unique.txt', index=False)


原始数据量: 8304718行
去重后的数据量: 7295027行


 ### 2、查看type和status字段有多少不一样的取值

In [1]:
import pandas as pd

# 加载数据
data_path = '流水数据/combined_data_unique.txt'
data = pd.read_csv(data_path)

# 获取status字段的所有唯一取值并统计每个唯一取值的出现次数
unique_status_values = data['status'].unique()
status_value_counts = data['status'].value_counts()

# 获取type字段的所有唯一取值并统计每个唯一取值的出现次数
unique_type_values = data['type'].unique()
type_value_counts = data['type'].value_counts()

# 打印status字段的结果
print(f"status字段有 {len(unique_status_values)} 个不同的取值。")
print("status各取值及其数量:")
print(status_value_counts)

# 打印type字段的结果
print(f"type字段有 {len(unique_type_values)} 个不同的取值。")
print("type各取值及其数量:")
print(type_value_counts)


status字段有 13 个不同的取值。
status各取值及其数量:
2000    6931692
2001     351935
2107       4794
1001       2954
2102       1938
2104       1073
0           301
2101        192
1002         95
2108         29
2109         12
2106          7
2002          5
Name: status, dtype: int64
type字段有 3 个不同的取值。
type各取值及其数量:
30    7290543
10       3373
11       1111
Name: type, dtype: int64


### 3、 只保留type为30和status为2000的字段，然后删除包含ctime的三个字段

In [3]:
import pandas as pd

# 加载数据
data_path = '流水数据/combined_data_unique.txt'
data = pd.read_csv(data_path)

# 显示原始数据大小
original_size = data.shape[0]
print(f"原始数据大小：{original_size} 行")

# 统计并过滤status字段
status_not_2000_count = data[data['status'] != 2000].shape[0]
data = data[data['status'] == 2000]
print(f"status不为2000的行有：{status_not_2000_count} 行")
print(f"删除status不为2000后的数据大小：{data.shape[0]} 行")

# 统计并过滤type字段
type_30_count = data[data['type'] == 30].shape[0]
data = data[data['type'] == 30]
print(f"type字段为30的行有：{type_30_count} 行")
print(f"删除type字段不为30的行后的数据大小：{data.shape[0]} 行")

# 删除不必要的字段
data.drop(['type', 'status', 'ctime'], axis=1, inplace=True)

# 保存清洗后的数据
output_file_path = '流水数据/clean.txt'
data.to_csv(output_file_path, index=False)
print(f"数据处理完成，并保存到 '{output_file_path}'。")

# 重新加载清洗后的数据
data = pd.read_csv(output_file_path)

# 检查每个字段是否包含 \N
contains_N_per_column = data.apply(lambda x: x.astype(str).str.contains(r'\\N'))

# 统计包含 \N 的情况
count_N_per_column = contains_N_per_column.sum()
total_N_rows = contains_N_per_column.any(axis=1).sum()
print(f"包含 '\\N' 的总行数: {total_N_rows}")
print("每个字段包含 '\\N' 的个数:")
print(count_N_per_column[count_N_per_column > 0])

# 删除包含 \N 的行
data_cleaned = data[~contains_N_per_column.any(axis=1)]
data_cleaned.to_csv(output_file_path, index=False)

# 输出删除 \N 后的数据大小
original_size = data.shape[0]
new_size = data_cleaned.shape[0]
print(f"原始数据行数: {original_size}")
print(f"删除包含 '\\N' 的行后的数据行数: {new_size}")


原始数据大小：7295027 行
status不为2000的行有：363335 行
删除status不为2000后的数据大小：6931692 行
type字段为30的行有：6927255 行
删除type字段不为30的行后的数据大小：6927255 行
数据处理完成，并保存到 '流水数据/clean.txt'。
包含 '\N' 的总行数: 11261
每个字段包含 '\N' 的个数:
paid_amount    11261
dtype: int64
原始数据行数: 6927255
删除包含 '\N' 的行后的数据行数: 6915994


#### 为什么用的是pt而不是ctime，因为经过检查，只有0.15%的ctime有出现第二遍，以这个来识别“天”有点没有意义。但是ctime是肯定有重复项的。（显然，一天不能只有一单吧。。。）

In [ ]:
import pandas as pd

# 加载数据
data_path = '流水数据/clean.txt'
data = pd.read_csv(data_path)

# 统计每个 ctime 出现的次数
ctime_counts = data['pt'].value_counts()

# 筛选出出现次数超过一次的 ctime
multiple_ctimes = ctime_counts[ctime_counts > 1]

# 计算出现多次的 ctime 数量
multiple_ctimes_count = multiple_ctimes.sum()

# 计算总数据条数
total_data_count = data.shape[0]

# 计算占总数据的比例
multiple_ctimes_percentage = (multiple_ctimes_count / total_data_count) * 100

print(f"出现不止一次的 pt 数量: {multiple_ctimes_count}")
print(f"占总数据比例: {multiple_ctimes_percentage:.2f}%")


出现不止一次的 pt 数量: 6915994
占总数据比例: 100.00%


In [4]:
import pandas as pd

# 加载数据
stream_data_path = '流水数据/clean.txt'
train_both_path = '训练1/merchant_in_both_datasets.txt'
train_combined_path = '训练1/merchant_only_in_combined.txt'
test_both_path = '测试1/test_merchant_in_both_datasets.txt'
test_combined_path = '测试1/test_merchant_only_in_combined.txt'

stream_data = pd.read_csv(stream_data_path)
train_both = pd.read_csv(train_both_path)
train_combined = pd.read_csv(train_combined_path)
test_both = pd.read_csv(test_both_path)
test_combined = pd.read_csv(test_combined_path)

# 合并四个文件中的merchant_id为一个集合
merchant_ids = pd.concat([train_both['merchant_id'], train_combined['merchant_id'], 
                          test_both['merchant_id'], test_combined['merchant_id']]).unique()

# 保留流水数据中的merchant_id如果出现在任意一个文件中
stream_data_cleaned = stream_data[stream_data['merchant_id'].isin(merchant_ids)]

# 保存处理后的数据
stream_data_cleaned.to_csv(stream_data_path, index=False)

# 打印信息
print(f"原始流水数据行数: {len(stream_data)}")
print(f"处理后的流水数据行数: {len(stream_data_cleaned)}")


原始流水数据行数: 6915994
处理后的流水数据行数: 3422391


In [5]:
import pandas as pd

# 文件路径
train_both_path = '训练1/merchant_in_both_datasets.txt'
train_combined_path = '训练1/merchant_only_in_combined.txt'
test_both_path = '测试1/test_merchant_in_both_datasets.txt'
test_combined_path = '测试1/test_merchant_only_in_combined.txt'
clean_data_path = '流水数据/clean.txt'

# 读取流水数据中的 merchant_id
clean_data = pd.read_csv(clean_data_path)
clean_merchant_ids = set(clean_data['merchant_id'].unique())

# 读取并检查文件中的 merchant_id
def check_merchant_ids(file_path):
    data = pd.read_csv(file_path)
    merchant_ids = set(data['merchant_id'].unique())
    total_ids = len(merchant_ids)
    missing_ids = merchant_ids - clean_merchant_ids
    missing_count = len(missing_ids)
    return total_ids, missing_count

# 检查每个文件
train_both_total, train_both_missing = check_merchant_ids(train_both_path)
train_combined_total, train_combined_missing = check_merchant_ids(train_combined_path)
test_both_total, test_both_missing = check_merchant_ids(test_both_path)
test_combined_total, test_combined_missing = check_merchant_ids(test_combined_path)

# 输出结果
print(f"train_both_his2.txt: Total Merchant IDs: {train_both_total}, Missing in clean.txt: {train_both_missing}")
print(f"train_combined_with_merchant_info_with_store_count.txt: Total Merchant IDs: {train_combined_total}, Missing in clean.txt: {train_combined_missing}")
print(f"test_both_his2.txt: Total Merchant IDs: {test_both_total}, Missing in clean.txt: {test_both_missing}")
print(f"test_combined_with_merchant_info_with_store_count.txt: Total Merchant IDs: {test_combined_total}, Missing in clean.txt: {test_combined_missing}")


train_both_his2.txt: Total Merchant IDs: 166, Missing in clean.txt: 0
train_combined_with_merchant_info_with_store_count.txt: Total Merchant IDs: 942, Missing in clean.txt: 0
test_both_his2.txt: Total Merchant IDs: 67, Missing in clean.txt: 0
test_combined_with_merchant_info_with_store_count.txt: Total Merchant IDs: 421, Missing in clean.txt: 0


In [6]:
import pandas as pd

# 文件路径
clean_data_path = '流水数据/clean.txt'
train_both_path = '训练1/merchant_in_both_datasets.txt'
train_combined_path = '训练1/merchant_only_in_combined.txt'
test_both_path = '测试1/test_merchant_in_both_datasets.txt'
test_combined_path = '测试1/test_merchant_only_in_combined.txt'

# 读取流水数据中的 merchant_id
clean_data = pd.read_csv(clean_data_path)
clean_merchant_ids = set(clean_data['merchant_id'].unique())
print(f"流水数据 'clean.txt' 中不同的 merchant_id 数量: {len(clean_merchant_ids)}")

# 读取并汇总所有文件中的 merchant_id
files = [train_both_path, train_combined_path, test_both_path, test_combined_path]
all_merchant_ids = set()

for file in files:
    data = pd.read_csv(file)
    all_merchant_ids.update(data['merchant_id'].unique())

print(f"四个文件中不同的 merchant_id 总数: {len(all_merchant_ids)}")


流水数据 'clean.txt' 中不同的 merchant_id 数量: 1586
四个文件中不同的 merchant_id 总数: 1586


: 